<a href="https://colab.research.google.com/github/Kashish-x1/text-analysis-project/blob/main/text_analysis_based_on_NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install beautifulsoup4 nltk

In [19]:
import nltk
nltk.download('punkt_tab')      # Breaks text into sentences and words
nltk.download('stopwords')  # Gives us a list of boring words like "the"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
!ls /content/

Input.xlsx				   Netclan20241021.txt	Netclan20241028.txt
MasterDictionary			   Netclan20241022.txt	Netclan20241029.txt
MasterDictionary-20250325T135116Z-001.zip  Netclan20241023.txt	sample_data
Netclan20241017.txt			   Netclan20241024.txt	StopWords
Netclan20241018.txt			   Netclan20241025.txt	StopWords-20250325T135103Z-001.zip
Netclan20241019.txt			   Netclan20241026.txt
Netclan20241020.txt			   Netclan20241027.txt


In [14]:
!unzip -o /content/StopWords-20250325T135103Z-001.zip -d /content/

Archive:  /content/StopWords-20250325T135103Z-001.zip
  inflating: /content/StopWords/StopWords_Generic.txt  
  inflating: /content/StopWords/StopWords_Geographic.txt  
  inflating: /content/StopWords/StopWords_Auditor.txt  
  inflating: /content/StopWords/StopWords_Currencies.txt  
  inflating: /content/StopWords/StopWords_Names.txt  
  inflating: /content/StopWords/StopWords_GenericLong.txt  
  inflating: /content/StopWords/StopWords_DatesandNumbers.txt  


In [15]:
!unzip -o /content/MasterDictionary-20250325T135116Z-001.zip -d /content/

Archive:  /content/MasterDictionary-20250325T135116Z-001.zip
  inflating: /content/MasterDictionary/negative-words.txt  
  inflating: /content/MasterDictionary/positive-words.txt  


In [16]:
import pandas as pd        # Reads Excel
import requests           # Downloads webpages
from bs4 import BeautifulSoup  # Finds article parts

# Load the list of URLs
df = pd.read_excel('/content/Input.xlsx')

# Go through each of the 147 rows
for index, row in df.iterrows():
    url_id = row['URL_ID']  # Get the ID (e.g., "Netclan20241017")
    url = row['URL']        # Get the link (e.g., "https://...")
    try:
        # Download the webpage
        response = requests.get(url, timeout=10)  # Wait 10 seconds max
        response.raise_for_status()  # Make sure it worked
        soup = BeautifulSoup(response.content, 'html.parser')  # Understand the webpage

        # Find the title
        title = soup.find('h1', class_='entry-title')  # Look for <h1 class="entry-title">
        if title:  # If found
            title_text = title.get_text(strip=True)  # Get just the text
        else:
            title_text = "No Title Found"  # If not found

        # Find the article text
        content = soup.find('div', class_='td-post-content')  # Look for <div class="td-post-content">
        if content:  # If found
            paragraphs = content.find_all('p')  # Get all <p> tags inside
            article_text = "\n\n".join(p.get_text(strip=True) for p in paragraphs)  # Combine paragraphs
        else:
            article_text = ""  # If not found

        # Put title and text together
        full_text = title_text + "\n\n" + article_text

        # Save to a file
        with open(f'/content/{url_id}.txt', 'w', encoding='utf-8') as f:
            f.write(full_text)
        print(f"Saved {url_id}.txt")
    except Exception as e:
        # If something goes wrong (e.g., bad link)
        print(f"Error with {url_id}: {e}")
        with open(f'/content/{url_id}.txt', 'w', encoding='utf-8') as f:
            f.write("Error: Could not get content")

Saved Netclan20241017.txt
Saved Netclan20241018.txt
Saved Netclan20241019.txt
Saved Netclan20241020.txt
Saved Netclan20241021.txt
Saved Netclan20241022.txt
Saved Netclan20241023.txt
Saved Netclan20241024.txt
Saved Netclan20241025.txt
Saved Netclan20241026.txt
Saved Netclan20241027.txt
Saved Netclan20241028.txt
Saved Netclan20241029.txt
Saved Netclan20241030.txt
Saved Netclan20241031.txt
Saved Netclan20241032.txt
Saved Netclan20241033.txt
Saved Netclan20241034.txt
Saved Netclan20241035.txt
Saved Netclan20241036.txt
Saved Netclan20241037.txt
Saved Netclan20241038.txt
Saved Netclan20241039.txt
Saved Netclan20241040.txt
Saved Netclan20241041.txt
Saved Netclan20241042.txt
Saved Netclan20241043.txt
Saved Netclan20241044.txt
Saved Netclan20241045.txt
Saved Netclan20241046.txt
Saved Netclan20241047.txt
Saved Netclan20241048.txt
Saved Netclan20241049.txt
Saved Netclan20241050.txt
Saved Netclan20241051.txt
Saved Netclan20241052.txt
Saved Netclan20241053.txt
Saved Netclan20241054.txt
Saved Netcla

In [20]:
import os
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize

# Load the word lists
stop_words = set()  # Words to ignore
for file in os.listdir('/content/StopWords'):
    with open(f'/content/StopWords/{file}', 'r', encoding='latin-1') as f:
        stop_words.update(line.strip().lower() for line in f)

positive_words = set()  # Happy words
negative_words = set()  # Sad words
with open('/content/MasterDictionary/positive-words.txt', 'r', encoding='latin-1') as f:
    positive_words.update(w.strip().lower() for w in f if w.strip().lower() not in stop_words)
with open('/content/MasterDictionary/negative-words.txt', 'r', encoding='latin-1') as f:
    negative_words.update(w.strip().lower() for w in f if w.strip().lower() not in stop_words)

# Function to analyze one text file
def analyze_text(file_path):
    # Open and read the file
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except:
        # If it fails, return all zeros
        return {
            'POSITIVE SCORE': 0, 'NEGATIVE SCORE': 0, 'POLARITY SCORE': 0, 'SUBJECTIVITY SCORE': 0,
            'AVG SENTENCE LENGTH': 0, 'PERCENTAGE OF COMPLEX WORDS': 0, 'FOG INDEX': 0,
            'AVG NUMBER OF WORDS PER SENTENCE': 0, 'COMPLEX WORD COUNT': 0, 'WORD COUNT': 0,
            'SYLLABLE PER WORD': 0, 'PERSONAL PRONOUNS': 0, 'AVG WORD LENGTH': 0
        }

    # Break the text into pieces
    sentences = sent_tokenize(text)  # List of sentences
    words = word_tokenize(text.lower())  # List of words, all lowercase
    cleaned_words = [w for w in words if w not in stop_words and w not in string.punctuation]
    word_count = len(cleaned_words)  # How many words after cleaning

    # Count happy and sad words
    positive_score = sum(1 for w in cleaned_words if w in positive_words)
    negative_score = sum(1 for w in cleaned_words if w in negative_words)
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (word_count + 0.000001)

    # How long are sentences?
    total_words = len(words)  # All words, even stop words
    sentence_count = len(sentences) if sentences else 1  # Avoid dividing by zero
    avg_sentence_length = total_words / sentence_count

    # Count syllables in words
    def count_syllables(word):
        word = word.lower()
        if word.endswith(('es', 'ed')):
            word = word[:-2]  # Remove "es" or "ed" endings
        vowels = 'aeiouy'
        count = sum(1 for char in word if char in vowels)
        return max(1, count)  # At least 1 syllable

    # Complex words have more than 2 syllables
    complex_words = sum(1 for w in cleaned_words if count_syllables(w) > 2)
    pct_complex_words = (complex_words / total_words) * 100 if total_words else 0
    fog_index = 0.4 * (avg_sentence_length + pct_complex_words)
    avg_words_per_sentence = total_words / sentence_count
    syllable_per_word = sum(count_syllables(w) for w in cleaned_words) / word_count if word_count else 0

    # Count personal pronouns
    pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I)) - len(re.findall(r'\bUS\b', text))

    # Average word length
    avg_word_length = sum(len(w) for w in cleaned_words) / word_count if word_count else 0

    # Return all 13 results
    return {
        'POSITIVE SCORE': positive_score, 'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score, 'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': avg_sentence_length, 'PERCENTAGE OF COMPLEX WORDS': pct_complex_words,
        'FOG INDEX': fog_index, 'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
        'COMPLEX WORD COUNT': complex_words, 'WORD COUNT': word_count,
        'SYLLABLE PER WORD': syllable_per_word, 'PERSONAL PRONOUNS': pronouns,
        'AVG WORD LENGTH': avg_word_length
    }

# Analyze all 147 files
results = []
for index, row in df.iterrows():
    url_id = row['URL_ID']
    analysis = analyze_text(f'/content/{url_id}.txt')
    result = {'URL_ID': url_id, 'URL': row['URL'], **analysis}
    results.append(result)
    print(f"Analyzed {url_id}")

Analyzed Netclan20241017
Analyzed Netclan20241018
Analyzed Netclan20241019
Analyzed Netclan20241020
Analyzed Netclan20241021
Analyzed Netclan20241022
Analyzed Netclan20241023
Analyzed Netclan20241024
Analyzed Netclan20241025
Analyzed Netclan20241026
Analyzed Netclan20241027
Analyzed Netclan20241028
Analyzed Netclan20241029
Analyzed Netclan20241030
Analyzed Netclan20241031
Analyzed Netclan20241032
Analyzed Netclan20241033
Analyzed Netclan20241034
Analyzed Netclan20241035
Analyzed Netclan20241036
Analyzed Netclan20241037
Analyzed Netclan20241038
Analyzed Netclan20241039
Analyzed Netclan20241040
Analyzed Netclan20241041
Analyzed Netclan20241042
Analyzed Netclan20241043
Analyzed Netclan20241044
Analyzed Netclan20241045
Analyzed Netclan20241046
Analyzed Netclan20241047
Analyzed Netclan20241048
Analyzed Netclan20241049
Analyzed Netclan20241050
Analyzed Netclan20241051
Analyzed Netclan20241052
Analyzed Netclan20241053
Analyzed Netclan20241054
Analyzed Netclan20241055
Analyzed Netclan20241056


In [21]:
# Save to CSV
output_df = pd.DataFrame(results)
output_df = output_df[['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                       'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                       'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']]
output_df.to_csv('/content/output.csv', index=False)

In [22]:
from google.colab import files
files.download('/content/output.csv')
!zip -r /content/text_files.zip /content/*.txt
files.download('/content/text_files.zip')

with open('/content/instructions.txt', 'w') as f:
    f.write('''Approach:
- Scraped 147 articles in Colab.
- Analyzed each for 13 metrics.
- Saved results in output.csv.

How to Run:
1. Upload Input.xlsx, StopWords, MasterDictionary.
2. Run the code.
3. Download output.csv and text_files.zip.

Dependencies:
- pandas, requests (pre-installed)
- beautifulsoup4, nltk (installed)''')
files.download('/content/instructions.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: content/Netclan20241017.txt (deflated 45%)
  adding: content/Netclan20241018.txt (deflated 60%)
  adding: content/Netclan20241019.txt (deflated 47%)
  adding: content/Netclan20241020.txt (deflated 59%)
  adding: content/Netclan20241021.txt (deflated 46%)
  adding: content/Netclan20241022.txt (deflated 45%)
  adding: content/Netclan20241023.txt (deflated 60%)
  adding: content/Netclan20241024.txt (deflated 47%)
  adding: content/Netclan20241025.txt (deflated 59%)
  adding: content/Netclan20241026.txt (deflated 65%)
  adding: content/Netclan20241027.txt (deflated 55%)
  adding: content/Netclan20241028.txt (deflated 58%)
  adding: content/Netclan20241029.txt (deflated 50%)
  adding: content/Netclan20241030.txt (deflated 53%)
  adding: content/Netclan20241031.txt (deflated 61%)
  adding: content/Netclan20241032.txt (deflated 47%)
  adding: content/Netclan20241033.txt (deflated 58%)
  adding: content/Netclan20241034.txt (deflated 45%)
  adding: content/Netclan20241035.txt (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>